# Gradient boosting with mixed estimators

Why only use decision trees?

## imports

In [1]:
from itertools import cycle, islice

In [2]:
from sklearn.base import BaseEstimator, RegressorMixin, clone
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.datasets import make_regression
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble._gb_losses import LeastSquaresError
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import BaggingRegressor
import numpy as np

## load data

In [3]:
# fake data
X, y = make_regression(1000, 50, n_informative=40, random_state=0, noise=1)
# or california housing data
X, y = fetch_california_housing(return_X_y=True)

In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0)

In [5]:
X.std(0)

array([1.89977569e+00, 1.25852527e+01, 2.47411320e+00, 4.73899376e-01,
       1.13243469e+03, 1.03857980e+01, 2.13590065e+00, 2.00348319e+00])

In [29]:
X.min(0)

array([   0.4999    ,    1.        ,    0.84615385,    0.33333333,
          3.        ,    0.69230769,   32.54      , -124.35      ])

In [30]:
X.max(0)

array([ 1.50001000e+01,  5.20000000e+01,  1.41909091e+02,  3.40666667e+01,
        3.56820000e+04,  1.24333333e+03,  4.19500000e+01, -1.14310000e+02])

In [6]:
y.min(), y.max(), y.mean(), y.std()

(0.14999, 5.00001, 2.068558169089147, 1.1539282040412253)

Purposefully no feature engineering

## custom code

no `set_params` on estimators implemented yet

#### light weight implementation

In [7]:
class ArbitraryBoostingRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, estimators, n_estimators=10, learning_rate=0.1):
        self.estimators = estimators
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        
        assert self.estimators
        
    def _estimator_cycle(self):
        for estimator in islice(cycle(self.estimators), self.n_estimators):
            yield clone(estimator)
            
    def fit(self, X, y, **fit_params):
        self.loss_ = LeastSquaresError(1)
        self.estimators_ = []
        self.errors_ = []
        y_resid = y

        self.init_ = self.loss_.init_estimator()
        y_resid = y - self.init_.fit(X, y, **fit_params).predict(X)

        for estimator in self._estimator_cycle():
            y_pred = estimator.fit(X, y_resid, **fit_params).predict(X)
            self.estimators_.append(estimator)
            y_resid = y_resid - y_pred * self.learning_rate
            self.errors_.append(mean_squared_error(y_resid, y_pred))
        return self
    
    def predict(self, X):
        y_pred = self.init_.predict(X)
        for estimator in self.estimators_:
            y_pred += self.learning_rate * estimator.predict(X)
        return y_pred

#### inheriting from GradientBoostingRegressor

In [8]:
class TreeWrapper:
    def __init__(self, obj):
        self.obj = obj
        
    @property
    def tree_(self):
        return self.obj
    
    def __getattr__(self, attr):
        return getattr(self.obj, attr)


class ArbitraryBoostingRegressor2(GradientBoostingRegressor):
    def __init__(
        self,
        estimators,
        *,
        loss='ls',
        learning_rate=0.1,
        n_estimators=10,
        subsample=1.0,
        criterion='friedman_mse',
        min_samples_split=2,
        min_samples_leaf=1,
        min_weight_fraction_leaf=0.0,
        max_depth=3,
        min_impurity_decrease=0.0,
        min_impurity_split=None,
        init=None,
        random_state=None,
        max_features=None,
        alpha=0.9,
        verbose=0,
        max_leaf_nodes=None,
        warm_start=False,
        presort='deprecated',
        validation_fraction=0.1,
        n_iter_no_change=None,
        tol=0.0001,
        ccp_alpha=0.0,
    ):
        super().__init__(
            loss=loss,
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            subsample=subsample,
            criterion=criterion,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            min_weight_fraction_leaf=min_weight_fraction_leaf,
            max_depth=max_depth,
            min_impurity_decrease=min_impurity_decrease,
            min_impurity_split=min_impurity_split,
            init=init,
            random_state=random_state,
            max_features=max_features,
            alpha=alpha,
            verbose=verbose,
            max_leaf_nodes=max_leaf_nodes,
            warm_start=warm_start,
            presort=presort,
            validation_fraction=validation_fraction,
            n_iter_no_change=n_iter_no_change,
            tol=tol,
            ccp_alpha=ccp_alpha,
        )
        self.estimators = estimators

    def _fit_stage(
        self, i, X, y, raw_predictions, sample_weight, sample_mask,
        random_state, X_idx_sorted, X_csc=None, X_csr=None):
        """Fit another stage of ``n_classes_`` trees to the boosting model. """

        assert sample_mask.dtype == np.bool
        loss = self.loss_
        original_y = y

        # Need to pass a copy of raw_predictions to negative_gradient()
        # because raw_predictions is partially updated at the end of the loop
        # in update_terminal_regions(), and gradients need to be evaluated at
        # iteration i - 1.
        raw_predictions_copy = raw_predictions.copy()

        for k in range(loss.K):
            if loss.is_multi_class:
                y = np.array(original_y == k, dtype=np.float64)

            residual = loss.negative_gradient(y, raw_predictions_copy, k=k,
                                              sample_weight=sample_weight)

            # induce regression tree on residuals
            tree = TreeWrapper(clone(next(islice(cycle(self.estimators), i, i+1))))

            if self.subsample < 1.0:
                # no inplace multiplication!
                sample_weight = sample_weight * sample_mask.astype(np.float64)

            X = X_csr if X_csr is not None else X
            tree.fit(X, residual)  # remaining arguments not supported

            # update tree leaves
            loss.update_terminal_regions(
                tree, X, y, residual, raw_predictions, sample_weight,
                sample_mask, learning_rate=self.learning_rate, k=k)

            # add tree to ensemble
            self.estimators_[i, k] = tree

        return raw_predictions

    def _raw_predict_init(self, X):
        """Check input and compute raw predictions of the init estimator."""
        self._check_initialized()
        if X.shape[1] != self.n_features_:
            raise ValueError("X.shape[1] should be {0:d}, not {1:d}.".format(
                self.n_features_, X.shape[1]))
        if self.init_ == 'zero':
            raw_predictions = np.zeros(shape=(X.shape[0], self.loss_.K),
                                       dtype=np.float64)
        else:
            raw_predictions = self.loss_.get_init_raw_predictions(
                X, self.init_).astype(np.float64)
        return raw_predictions

    def _raw_predict(self, X):
        """Return the sum of the trees raw predictions (+ init estimator)."""
        raw_predictions = self._raw_predict_init(X)
        for i in range(self.n_estimators):
            regr = self.estimators_[i][0]
            raw_predictions += self.learning_rate * regr.predict(X).reshape(-1, 1)
        return raw_predictions

## experiments

### fit a single model

#### linear regression

In [9]:
abr = ArbitraryBoostingRegressor([LinearRegression()])
abr.fit(X_train, y_train)
mean_squared_error(y_valid, abr.predict(X_valid))

0.6263952602050491

In [10]:
abr.errors_

[0.5273812806033222,
 0.5258319802944061,
 0.5245770470441843,
 0.5235605511115047,
 0.5227371894060342,
 0.5220702664246031,
 0.5215300588096439,
 0.5210924906415269,
 0.5207380604253522,
 0.5204509719502506]

#### linear + dt + knn

In [11]:
regr = ArbitraryBoostingRegressor([LinearRegression(), DecisionTreeRegressor(), KNeighborsRegressor()])
abr.fit(X_train, y_train)
mean_squared_error(y_valid, abr.predict(X_valid))

0.6263952602050491

In [12]:
abr.errors_

[0.5273812806033222,
 0.5258319802944061,
 0.5245770470441843,
 0.5235605511115047,
 0.5227371894060342,
 0.5220702664246031,
 0.5215300588096439,
 0.5210924906415269,
 0.5207380604253522,
 0.5204509719502506]

### cross validate approach

compare the 2 implementations

#### dt

In [13]:
%%time
regr = ArbitraryBoostingRegressor([DecisionTreeRegressor()])
scores = cross_val_score(regr, X, y, cv=5, scoring='neg_mean_squared_error')
print(-np.mean(scores))
print(scores)

0.6723538160141442
[-0.60034349 -0.5952506  -0.69019247 -0.71269834 -0.76328417]
CPU times: user 9.03 s, sys: 39 ms, total: 9.06 s
Wall time: 8.52 s


In [14]:
%%time
regr = ArbitraryBoostingRegressor2([DecisionTreeRegressor()])
scores = cross_val_score(regr, X, y, cv=5, scoring='neg_mean_squared_error')
print(-np.mean(scores))
print(scores)

0.6722669141379157
[-0.58848153 -0.59639745 -0.70226855 -0.71117485 -0.76301219]
CPU times: user 8.53 s, sys: 43.8 ms, total: 8.57 s
Wall time: 8.57 s


use default GradientBoostingRegressor hyper params

In [15]:
%%time
dt = DecisionTreeRegressor(
    criterion='friedman_mse',
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_depth=3,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
)
regr = ArbitraryBoostingRegressor2([dt])
scores = cross_val_score(regr, X, y, cv=5, scoring='neg_mean_squared_error')
print(-np.mean(scores))
print(scores)

0.7011773270758519
[-0.62248244 -0.63549276 -0.74758768 -0.78483192 -0.71549184]
CPU times: user 1.74 s, sys: 40 µs, total: 1.74 s
Wall time: 1.74 s


### lr

In [16]:
%%time
regr = ArbitraryBoostingRegressor([LinearRegression()])
scores = cross_val_score(regr, X, y, cv=5, scoring='neg_mean_squared_error')
print(-np.mean(scores))
print(scores)

0.6535256771574559
[-0.56811679 -0.69759173 -0.75929642 -0.65384625 -0.5887772 ]
CPU times: user 595 ms, sys: 20 ms, total: 615 ms
Wall time: 157 ms


In [17]:
%%time
regr = ArbitraryBoostingRegressor2([LinearRegression()])
scores = cross_val_score(regr, X, y, cv=5, scoring='neg_mean_squared_error')
print(-np.mean(scores))
print(scores)

0.6535163904654909
[-0.5680934  -0.69759844 -0.75928304 -0.65383644 -0.58877063]
CPU times: user 536 ms, sys: 7.73 ms, total: 544 ms
Wall time: 137 ms


### knn

In [18]:
%%time
regr = ArbitraryBoostingRegressor([KNeighborsRegressor()])
scores = cross_val_score(regr, X, y, cv=5, scoring='neg_mean_squared_error')
print(-np.mean(scores))
print(scores)

1.2037599868343167
[-1.09356936 -1.03899702 -1.35285596 -1.18142013 -1.35195747]
CPU times: user 6.06 s, sys: 16 ms, total: 6.08 s
Wall time: 5.51 s


In [19]:
%%time
regr = ArbitraryBoostingRegressor2([KNeighborsRegressor()])
scores = cross_val_score(regr, X, y, cv=5, scoring='neg_mean_squared_error')
print(-np.mean(scores))
print(scores)

1.2037599868343167
[-1.09356936 -1.03899702 -1.35285596 -1.18142013 -1.35195747]
CPU times: user 5.4 s, sys: 32 ms, total: 5.43 s
Wall time: 5.43 s


#### lr + dt

In [20]:
%%time
regr = ArbitraryBoostingRegressor([LinearRegression(), dt])
scores = cross_val_score(regr, X, y, cv=5, scoring='neg_mean_squared_error')
print(-np.mean(scores))
print(scores)

0.6334502502318948
[-0.55735801 -0.63247047 -0.70369275 -0.67487158 -0.59885843]
CPU times: user 3.81 s, sys: 67.9 ms, total: 3.88 s
Wall time: 973 ms


In [21]:
%%time
regr = ArbitraryBoostingRegressor2([LinearRegression(), dt])
scores = cross_val_score(regr, X, y, cv=5, scoring='neg_mean_squared_error')
print(-np.mean(scores))
print(scores)

0.6334443270638227
[-0.55734265 -0.63247685 -0.70368564 -0.67486366 -0.59885283]
CPU times: user 3.79 s, sys: 48 ms, total: 3.84 s
Wall time: 960 ms


#### lr + dt + knn

In [22]:
%%time
regr = ArbitraryBoostingRegressor([LinearRegression(), dt, KNeighborsRegressor()])
scores = cross_val_score(regr, X, y, cv=5, scoring='neg_mean_squared_error')
print(-np.mean(scores))
print(scores)

0.7240332948253368
[-0.6353083  -0.70156824 -0.81329698 -0.74860433 -0.72138863]
CPU times: user 9.27 s, sys: 152 ms, total: 9.43 s
Wall time: 2.36 s


In [23]:
%%time
regr = ArbitraryBoostingRegressor2([LinearRegression(), dt, KNeighborsRegressor()])
scores = cross_val_score(regr, X, y, cv=5, scoring='neg_mean_squared_error')
print(-np.mean(scores))
print(scores)

0.7240273214247799
[-0.63529467 -0.70157353 -0.81329003 -0.7485971  -0.72138127]
CPU times: user 9.85 s, sys: 132 ms, total: 9.98 s
Wall time: 2.49 s


### benchmarks

#### dummy

In [24]:
%%time
regr = DummyRegressor()
scores = cross_val_score(regr, X, y, cv=5, scoring='neg_mean_squared_error')
print(-np.mean(scores))
print(scores)

1.3702550894247665
[-1.3064619  -1.19893373 -1.57273505 -1.25865506 -1.5144897 ]
CPU times: user 42.7 ms, sys: 34 µs, total: 42.8 ms
Wall time: 10.9 ms


#### vanilla gradient boosting

In [25]:
%%time
regr = GradientBoostingRegressor(n_estimators=10)
scores = cross_val_score(regr, X, y, cv=5, scoring='neg_mean_squared_error')
print(-np.mean(scores))
print(scores)

0.7011773270758519
[-0.62248244 -0.63549276 -0.74758768 -0.78483192 -0.71549184]
CPU times: user 2.26 s, sys: 7.96 ms, total: 2.26 s
Wall time: 1.75 s


#### random forest

In [26]:
%%time
regr = RandomForestRegressor(n_estimators=10)
scores = cross_val_score(regr, X, y, cv=5, scoring='neg_mean_squared_error')
print(-np.mean(scores))
print(scores)

0.4847291292478402
[-0.62378846 -0.38590051 -0.41904684 -0.48970468 -0.50520516]
CPU times: user 5.16 s, sys: 12 ms, total: 5.17 s
Wall time: 5.17 s


#### bagging + lr

In [27]:
%%time
regr = BaggingRegressor(LinearRegression(), n_estimators=10)
scores = cross_val_score(regr, X, y, cv=5, scoring='neg_mean_squared_error')
print(-np.mean(scores))
print(scores)

0.5587854379934888
[-0.49734083 -0.62139618 -0.63548575 -0.54329421 -0.49641022]
CPU times: user 962 ms, sys: 8.06 ms, total: 970 ms
Wall time: 249 ms


#### bagging + dt

should roughly equal rf

In [28]:
%%time
regr = BaggingRegressor(DecisionTreeRegressor(), n_estimators=10)
scores = cross_val_score(regr, X, y, cv=5, scoring='neg_mean_squared_error')
print(-np.mean(scores))
print(scores)

0.46562762669283
[-0.51150194 -0.37253359 -0.42157403 -0.51776368 -0.5047649 ]
CPU times: user 5.65 s, sys: 56 ms, total: 5.71 s
Wall time: 5.13 s


## Further ideas

- use a bandit approach to choose next estimator to fit, e.g. Thompson sampling